# Análise das despesas da prefeitura de João Pessoa

In [2]:
#Imports
%pip install pandas
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Read data
df2021 = pd.read_csv(filepath_or_buffer='./data/2021.csv', sep='|')
df2022 = pd.read_csv(filepath_or_buffer='./data/2022.csv', sep='|')
df2023 = pd.read_csv(filepath_or_buffer='./data/2023.csv', sep='|')
df2024 = pd.read_csv(filepath_or_buffer='./data/2024.csv', sep='|')

In [4]:
dtypes =[
    df2021.dtypes,
    df2022.dtypes,
    df2023.dtypes,
    df2024.dtypes
]

dtypes

[nume_empenho              int64
 ano_empenho               int64
 data_empenho             object
 unidade_orcamentaria     object
 tipo_movimentacao        object
 favorecido               object
 codi_unor                 int64
 valor_empenhado         float64
 valor_liquidado         float64
 valor_pago              float64
 saldo_pagar             float64
 dtype: object,
 nume_empenho              int64
 ano_empenho               int64
 data_empenho             object
 unidade_orcamentaria     object
 tipo_movimentacao        object
 favorecido               object
 codi_unor                 int64
 valor_empenhado         float64
 valor_liquidado         float64
 valor_pago              float64
 saldo_pagar             float64
 dtype: object,
 nume_empenho              int64
 ano_empenho               int64
 data_empenho             object
 unidade_orcamentaria     object
 tipo_movimentacao        object
 favorecido               object
 codi_unor                 int64
 valor_empe

#### Conversões e ajustes de dados

In [5]:
# concat all data frames
df = pd.concat([df2021, df2022, df2023, df2024])
df["ano_empenho"].unique()

array([2021, 2022, 2023, 2024], dtype=int64)

In [6]:
linhas_ano = df[["ano_empenho"]]

linhas_ano =  linhas_ano.groupby("ano_empenho").size()

linhas_ano

ano_empenho
2021    28421
2022    29567
2023    31422
2024     9803
dtype: int64

In [7]:
# convert to date
df['data_empenho'] = pd.to_datetime(df['data_empenho'])
df.dtypes

nume_empenho                     int64
ano_empenho                      int64
data_empenho            datetime64[ns]
unidade_orcamentaria            object
tipo_movimentacao               object
favorecido                      object
codi_unor                        int64
valor_empenhado                float64
valor_liquidado                float64
valor_pago                     float64
saldo_pagar                    float64
dtype: object

### Dicionário de variáveis
|Nome|Tipo|Descrição|
|:-:|:-:|:-:|
| nume_empenho             | int64|Número do empenho a qual amovimentação se refere.|
| ano_empenho              | int64|Ano do empenho a qual a movimentação se refere.|
| data_empenho             |datetime64[ns]|Dia mês e ano do empenho a qual a movimentação se refere.|
| unidade_orcamentaria     |object|Descrição da unidade orçamentária.|
| tipo_movimentacao        |object|Descrição do tipo de movimentação|
| favorecido               |object|Descrição do favorecido|
| codi_unor                | int64|Código da unidade orçamentária.|
| valor_empenhado         |float64|Valor empenhado pelo empenho associado a movimentação.|
| valor_liquidado         |float64|Valor efetuado.|
| valor_pago              |float64|Valor pago.|
| saldo_pagar             |float64|Saldo a pagar.|

In [8]:
df.head(3)

,nume_empenho,ano_empenho,data_empenho,unidade_orcamentaria,tipo_movimentacao,favorecido,codi_unor,valor_empenhado,valor_liquidado,valor_pago,saldo_pagar
0,1,2021,2021-01-04,ASSESSORIA SUPERIOR,EMISSAO DE EMPENHO,MJV SERVICOS DE ESCRITORIOS COMPARTILHADOS LTDA,20101,102275.00,102275.00,102275.00,0.0
1,1,2021,2021-01-15,RECURSOS SOB A SUPERVISAO DA SECRETARIA DAS FI...,EMISSAO DE EMPENHO,"MINISTERIO DA FAZENDA, ECONOMIA E PLANEJAMENTO",160102,596099.69,596099.69,596099.69,0.0
2,1,2021,2021-01-15,FUNDO MUNICIPAL DE CULTURA,EMISSAO DE EMPENHO,BANCO DO BRASIL S/A,100301,0.00,0.00,0.00,0.0


## Lista de Unidades Orçamentárias

In [9]:
unidades_orcamentarias = pd.DataFrame(df['unidade_orcamentaria'].unique(),columns=["unidades_orcamentarias"]).sort_values("unidades_orcamentarias")
unidades_orcamentarias.head(3)

,unidades_orcamentarias
81,ASSESSORIA MILITAR
0,ASSESSORIA SUPERIOR
60,BANCO CIDADAO


## Lista de tipos de movimentação

In [10]:
tipos_movimentacoes = pd.DataFrame(df['tipo_movimentacao'].unique(),columns=["tipos_movimentacoes"]).sort_values("tipos_movimentacoes")
tipos_movimentacoes.head(3)

,tipos_movimentacoes
0,EMISSAO DE EMPENHO


## Lista de tipos de favorecidos

In [11]:
favorecidos = pd.DataFrame(df['favorecido'].unique(),columns=["favorecidos"]).sort_values("favorecidos")
favorecidos.head(3)

,favorecidos
18686,1000MEDIC DISTRIBUIDORA IMPORTADORA EXPORTADOR...
786,1001 SERVICOS ENGENHARIA E CONSTRUCAO LTDA
18220,15.340.956 MARIA COSTA MACIEL SILVA


## Valor total por empenho

In [12]:
valor_empenho = df[["nume_empenho", "valor_empenhado","valor_liquidado","valor_pago","saldo_pagar" ]]

valor_empenho =  valor_empenho.groupby("nume_empenho").sum()

valor_empenho

,valor_empenhado,valor_liquidado,valor_pago,saldo_pagar
nume_empenho,,,,
1,52016128.45,42208136.21,41934430.88,273705.30
2,33985217.01,23878963.58,19833037.91,4045925.63
3,34437414.42,33308173.92,26920724.34,6387449.58
4,18246155.74,17585125.48,16123207.10,1461918.37
5,48365282.56,42091744.89,49148685.65,-7056940.79
...,...,...,...,...
8498,5000.00,5000.00,4230.81,769.19
8499,695349.00,695349.00,600378.50,94970.50
8500,1748.84,1748.84,1748.84,0.00


## Valor total empenhado por ano

In [13]:
empenho_ano = df[["ano_empenho", "valor_empenhado","valor_liquidado","valor_pago","saldo_pagar" ]]

empenho_ano =  empenho_ano.groupby("ano_empenho").sum()

empenho_ano

,valor_empenhado,valor_liquidado,valor_pago,saldo_pagar
ano_empenho,,,,
2021,2.581685e+09,2.291340e+09,1.903027e+09,3.883131e+08
2022,3.151151e+09,2.830327e+09,2.437271e+09,3.930563e+08
2023,3.671847e+09,3.427864e+09,4.319691e+09,-8.918268e+08
2024,1.324028e+09,7.660640e+08,7.822100e+08,-1.614604e+07


## Valor total por unidade orçamentária

In [14]:
valor_unidades_orcamentarias = df[["unidade_orcamentaria", "valor_empenhado","valor_liquidado","valor_pago","saldo_pagar" ]]

valor_unidades_orcamentarias =  valor_unidades_orcamentarias.groupby("unidade_orcamentaria").sum()

valor_unidades_orcamentarias.sort_values(by="valor_empenhado",ascending=False)

,valor_empenhado,valor_liquidado,valor_pago,saldo_pagar
unidade_orcamentaria,,,,
FUNDO MUNICIPAL DE SAUDE,3.434592e+09,3.060114e+09,3.770732e+09,-7.106176e+08
SECRETARIA MUNICIPAL DE EDUCACAO E CULTURA-GABINETE DO SECRETARIO,1.623818e+09,1.330292e+09,1.210931e+09,1.193607e+08
INSTITUTO DE PREVIDENCIA DO MUNICIPIO - IPM,1.120994e+09,1.120665e+09,8.569576e+08,2.637070e+08
SUPERINTENDENCIA DE LIMPEZA URBANA - EMLUR,5.981059e+08,5.444643e+08,4.711396e+08,7.332465e+07
RECURSOS SOB A SUPERVISAO DA SECRETARIA DA ADMINISTRACAO,5.169522e+08,4.117595e+08,4.797412e+08,-6.798168e+07
...,...,...,...,...
OUVIDORIA GERAL DO MUNICIPIO,2.391410e+03,2.391410e+03,2.214560e+03,1.768500e+02
DIRETORIA ADM FIN - SECRETARIA DE INFRAESTRUTURA,1.615690e+03,1.615690e+03,1.615690e+03,0.000000e+00
DIRETORIA DE PROGRAMACAO ORCAMENTARIA,8.970000e+02,8.970000e+02,8.970000e+02,0.000000e+00


## Valor total por favorecido

In [15]:
valor_favorecidos = df[["favorecido", "valor_empenhado","valor_liquidado","valor_pago","saldo_pagar" ]]

valor_favorecidos = valor_favorecidos.groupby("favorecido").sum()

valor_favorecidos.sort_values(by="valor_empenhado", ascending=False)

,valor_empenhado,valor_liquidado,valor_pago,saldo_pagar
favorecido,,,,
SECRETARIA DE FINANCAS DO MUNICIPIO (SEFIN),1.924296e+09,1.924292e+09,1.454360e+09,4.699314e+08
FUNDO MUNICIPAL DE SAUDE,1.678694e+09,1.678694e+09,1.326502e+09,3.521922e+08
IPM-INSTITUTO DE PREVIDENCIA DO MUNICIPIO,1.405689e+09,1.399575e+09,1.253766e+09,1.458087e+08
INSTITUTO NACIONAL DO SEGURO SOCIAL INSS,4.927875e+08,4.646236e+08,1.208117e+09,-7.434930e+08
EXEMPLAR SERVICE E LIMPEZA LTDA,2.861965e+08,2.648036e+08,2.160477e+08,4.875591e+07
...,...,...,...,...
ROSEMARY ROZENO GOMEZ,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
ELIZABETH FABRICIO DA SILVA,0.000000e+00,3.500000e+02,0.000000e+00,3.500000e+02
TERESINHA CAVALCANTI DA SILVA COSTA,0.000000e+00,3.000000e+02,3.000000e+02,0.000000e+00


## Valor pago por trimestre em cada ano

In [16]:
empenho_trimestre = df[["data_empenho", "valor_pago" ]]

empenho_trimestre =  empenho_trimestre.groupby(empenho_trimestre["data_empenho"].dt.to_period("Q"))["valor_pago"].sum()

empenho_trimestre = pd.DataFrame(empenho_trimestre, columns=["valor_pago"])

empenho_trimestre

,valor_pago
data_empenho,
2021Q1,4.746501e+08
2021Q2,4.632089e+08
2021Q3,4.668986e+08
2021Q4,4.982688e+08
2022Q1,6.455455e+08
2022Q2,5.726017e+08
2022Q3,6.165109e+08
2022Q4,6.026131e+08
2023Q1,1.653865e+09


## Valor pago por mês em cada ano

In [18]:
empenho_mes= df[["data_empenho", "valor_pago" ]]

empenho_mes=  empenho_mes.groupby([ empenho_mes["data_empenho"].dt.month,empenho_mes["data_empenho"].dt.year])["valor_pago"].sum()

empenho_mes = empenho_mes.to_frame()


empenho_mes

valor_pago
data_empenho data_empenho              
1            2021          2.428232e+08
             2022          3.332823e+08
             2023          1.052824e+09
             2024          3.308314e+08
2            2021          1.095496e+08
             2022          1.469276e+08
             2023          2.983594e+08
             2024          2.679091e+08
3            2021          1.222774e+08
             2022          1.653356e+08
             2023          3.026817e+08
             2024          1.829330e+08
4            2021          1.822667e+08
             2022          1.947999e+08
             2023          2.268383e+08
             2024          5.366494e+05
5            2021          1.213301e+08
             2022          1.628938e+08
             2023          2.383388e+08
6            2021          1.596122e+08
             2022          2.149080e+08
             2023          3.734039e+08
7            2021          1.828115e+08
             2022          2.441712e+08
             2023          2.383938e+08
8            2021          1.495759e+08
             2022          1.929277e+08
             2023          3.096677e+08
9            2021          1.345112e+08
             2022          1.794120e+08
             2023          2.121749e+08
10           2021          1.491420e+08
             2022          1.633936e+08
             2023          2.768883e+08
11           2021          1.290918e+08
             2022          1.735709e+08
             2023          2.889000e+08
12           2021          2.200350e+08
             2022          2.656486e+08
             2023          5.012195e+08